# Tutorial 2 - Extracción de noticias en medios de prensa internacionales

Ejemplo aplicado al medio: https://www1.folha.uol.com.br/ultimas-noticias/

sintáxis del lenguaje XPath: https://www.w3schools.com/xml/xpath_syntax.asp

## Parte 1: Crawling

Recopilación de las URLs que contienen noticias de prensa

### 1. URL _seed_

In [5]:
SEED_CRAWL = 'https://www1.folha.uol.com.br/ultimas-noticias'

Para iniciar el proceso de crawling, necesitamos una página inicial que llamamos 'SEED'. Es la página que contiene una lista de enlaces hacia noticias.

### 2. Class _Crawler_

La clase _Crawler_ se encarga de enviar una consulta HTTP (GET) a la pagina SEED.  Luego analiza la respuesta HTML para obtener los enlaces HTML.

Se requiere definir la ruta HTML para obtener los enlaces que corresponden a las noticias de prensa.

In [29]:
QUERY_URL = "//div[@class='c-headline__content']/a/@href"

Para consultar y analizar la pagina SEED, se utiliza una serie de librerias y se selecciona al azar un navegador web.

In [30]:
import random
import requests
from requests_html import HTMLSession
from requests_html import HTML
import os
import json
import validators

In [31]:
class Crawler:
    
    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
]

    #ruta XPATH hacia los enlaces que queremos recopilar
    query_extract_all_href=QUERY_URL

    #variable que permite listar todos los enlaces recopilado
    all_json_to_send = []
    
    def _init_(self):
        pass
    
    def start_request(self,url):
        session = HTMLSession()
        header = dict()
        header['user-agent'] = random.choice(self.USER_AGENT_LIST)
        response = session.get(url,headers=header)        
        return response
    
    def parse(self, response):
        self.all_json_to_send = []
        all_extracted_href = response.html.xpath(self.query_extract_all_href)
        for href in all_extracted_href:
            json_to_send= dict()
            json_to_send["url"]=href
            self.all_json_to_send.append(json_to_send)

### 3. Crawling y parsing

In [32]:
crawler = Crawler()

response = crawler.start_request(SEED_CRAWL)

In [33]:
response.html.html

'\n\n<!doctype html>\n<html lang="pt-BR" data-version="prod@72e21b6f">\n  <head>\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <meta property="desliga:paywall" content="false">\r\n    <title>Últimas notícias: Em cima da hora | Folha</title>\n  <meta name="description" content="Acompanhe as últimas notícias na Folha! Atualizações do Brasil e do Mundo sobre Mercado, Esporte, Política, Famosos, Saúde, Cultura e vários temas. Acesse!">\n  <meta name="keywords" content="folha">\n    <meta http-equiv="Content-Security-Policy" content="upgrade-insecure-requests">\n  \n        <link rel="canonical" href="https://www1.folha.uol.com.br/ultimas-noticias">\n\n      \n  \n  \n  <!-- Open Graph Meta Tags -->\n      <meta property="og:description" content="Acompanhe as últimas notícias na Folha! Atualizações do Brasil e do Mundo sobre Mercado, Esporte, Política, Famosos, Saúde, Cultura e vários temas. Acesse!">\n          <meta property="og:image" content

In [34]:
crawler.parse(response)

In [36]:
crawler.all_json_to_send

[{'url': 'https://pitacocultural.blogfolha.uol.com.br/?p=1014'},
 {'url': 'https://www1.folha.uol.com.br/mercado/2020/10/tesouro-selic-repete-2002-e-tem-rendimento-negativo.shtml'},
 {'url': 'https://www1.folha.uol.com.br/esporte/2020/10/empresario-movimenta-milhoes-no-mercado-da-bola-em-plena-pandemia.shtml'},
 {'url': 'https://www1.folha.uol.com.br/ilustrada/2020/10/dancarinos-celebram-martin-luther-king-com-o-belissimo-sunshine.shtml'},
 {'url': 'https://www1.folha.uol.com.br/poder/2020/10/candidatos-kalil-em-bh-e-pastor-isidorio-em-salvador-participam-da-sabatina-folhauol.shtml'},
 {'url': 'https://cienciafundamental.blogfolha.uol.com.br/?p=225'},
 {'url': 'https://bompracachorro.blogfolha.uol.com.br/?p=12880'},
 {'url': 'https://bompracachorro.blogfolha.uol.com.br/?p=12885'},
 {'url': 'https://gatices.blogfolha.uol.com.br/?p=2781'},
 {'url': 'https://agora.folha.uol.com.br/esporte/2020/10/caneladas-do-vitao-desrespeito-reiterado-a-data-fifa-piora-o-que-ja-e-armagedonico.shtml'},
 

### 4. Testing

Para verificar que el crawler funciona, implementamos dos tests:

In [37]:
#el crawler debe devolver una lista no vacia
assert(len(crawler.all_json_to_send)>0) 

In [38]:
#lo que devuelve el crawler deben ser URLs correctas
assert validators.url(crawler.all_json_to_send[0]["url"])

## Parte 2: Scraping

Recopilación del contenido y de metadatos (título, fecha) de las noticias de prensa

### 1. URL _seed_ 

In [43]:
SEED_SCRAP = 'https://www1.folha.uol.com.br/mercado/2020/10/tesouro-selic-repete-2002-e-tem-rendimento-negativo.shtml'

Para iniciar el proceso de scraping, necesitamos una página inicial que llamamos 'SEED'. Es la página que contiene una noticia.


### 2. Class _Scraper_

La clase _Scraper_ se encarga de enviar una consulta HTTP (GET) a la pagina SEED. Luego analiza la respuesta HTML para obtener el contenido de la noticia y sus metadatos.

Se requiere definir la ruta HTML para obtener los datos que nos interesan.

In [88]:
QUERY_TITLE = "//h1[@class='c-content-head__title']"
QUERY_CONTENT =  "//div[@class='c-news__body']//p"
QUERY_DATE = "//time[@class='c-more-options__published-date']/@datetime"

In [89]:
import json
import w3lib.html
import random
import os
import datetime
import requests
import string
import calendar
import html
import locale 
from requests_html import HTMLSession

In [90]:
class Scraper:
    
    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]

    query_extract_title = QUERY_TITLE
    query_extract_text =  QUERY_CONTENT
    query_extract_date = QUERY_DATE

    def _init_(self):
        pass

    #no cambiar    
    def start_request(self,url):
        headers = {'user-agent':random.choice(self.USER_AGENT_LIST) }
        session = HTMLSession()
        response = session.get(url,headers=headers)        
        return response
    
    #no cambiar
    def clean_text(self, html_text):
        text_without_tags = w3lib.html.remove_tags(html_text)
        text_without_escape_chars = w3lib.html.replace_escape_chars(text_without_tags)
        text_without_escape_chars = html.unescape(text_without_escape_chars)
        text_without_whitespace = text_without_escape_chars.strip()
        return text_without_whitespace

    #no cambiar
    def replace_month_name_to_number(self,date_string):
        lower_string = date_string.lower()
        for month_id in range(1, 13):
            if(lower_string.find(calendar.month_name[month_id].lower())!=-1):
                if(month_id>=10):
                    return (lower_string.replace(calendar.month_name[month_id].lower(),str(month_id)))
                else:
                    return (lower_string.replace(calendar.month_name[month_id].lower(),"0"+str(month_id)))
            if(lower_string.find(calendar.month_abbr[month_id].lower())!=-1):
                if(month_id>=10):
                    return (lower_string.replace(calendar.month_abbr[month_id].lower(),str(month_id)))
                else:
                    return (lower_string.replace(calendar.month_abbr[month_id].lower(),"0"+str(month_id)))
                    
    #se puede cambiar
    def format_date(self,date):
        return(date.split(" ")[0])
    
    #se puede cambiar
    def parse(self, response):
        self.news_text = []
        self.message_to_send = []
        self.scraped_url = response.url
        self.news_title = response.html.xpath(self.query_extract_title)[0].text
        self.date = response.html.xpath(self.query_extract_date)[0]
        #print(self.date)

        self.news_title = self.clean_text(self.news_title)
        self.date = self.clean_text(self.date)
        self.date = self.format_date(self.date)
        
        news_text_extracted_response = response.html.xpath(self.query_extract_text)
        for part_of_body in news_text_extracted_response:
            stripped_html_tags = w3lib.html.remove_tags(part_of_body.text)
            text_cleaned = self.clean_text(stripped_html_tags)
            self.news_text.append(text_cleaned)  

### 3. Scraping

In [91]:
scraper = Scraper()
response =scraper.start_request(SEED_SCRAP)

In [92]:
scraper.parse(response)

In [93]:
scraper.news_title

'Tesouro Selic repete 2002 e tem rendimento negativo'

In [94]:
scraper.date

'2020-10-07'

In [95]:
scraper.news_text

['Nem a renda fixa escapou da volatilidade do mercado brasileiro nas últimas semanas. Títulos do Tesouro e fundos DI, que investem nestes tipos de ativo, encerraram setembro em queda, assustando investidores. É o caso dos títulos pós-fixados (LFTs), conhecidos como Tesouro Selic.',
 'As LFTs com vencimento de setembro de 2022 a setembro de 2026, por exemplo, caíram 0,04% a 1,01% no mês passado, respectivamente, segundo dados da Economatica. É a segunda vez que isso acontece.',
 '',
 '',
 'A primeira foi em maio de 2002, quando passou a valer a regra de marcação a mercado, contabilização dos papéis de um fundo com base em seu valor de mercado —antes, o investidor via o valor de aquisição dos títulos. A mudança assustou investidores e os levou a trocar fundos de renda fixa pela poupança, reduzindo a rentabilidade dos fundos, que tiveram que se desfazer de papéis para dar saída ao investidor. A grande pressão vendedora deixou a rentabilidade dos títulos naquele mês negativa.Além disso, as

### 4. Testing

In [96]:
# Una función que verifica que las fechas tienen el formato correcto YYYY-MM-DD
def validate(date_text):
    try:
        datetime.datetime.strptime(date_text, "%Y-%m-%d")
        return True
    except:
        print("La fecha no tiene el formato correcto, año-mes-dia")
        return False

In [97]:
#Verificamos que el scraper devuelve algo y que las fechas tienen un formato correcto
assert(len(scraper.news_title)>0)
assert(len(scraper.news_text)>0)
assert(validate(scraper.date))